In [15]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import urllib.request

In [16]:
# Download spacex_launch_dash.csv
url_csv = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
urllib.request.urlretrieve(url_csv, "spacex_launch_dash.csv")

# Download spacex_dash_app.py
url_py = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"
urllib.request.urlretrieve(url_py, "spacex_dash_app.py")

# Read the SpaceX launch data
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [17]:
# Create Dash app
app = dash.Dash(__name__)

In [18]:
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center', 'color': '#503D36','font-size': 40}),
    html.Br(),
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label':'All Sites', 'value':'ALL'},
            {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
            {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
            {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
            {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'}
        ],
        value='ALL',
        placeholder='Select a launch site',
        searchable=True
    ),
    html.Br(),
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=min_payload,
        max=max_payload,
        step=1000,
        marks={min_payload: str(min_payload), max_payload: str(max_payload)},
        value=[min_payload, max_payload]
    ),
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])


In [19]:
# Callback function for `site-dropdown`
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def update_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Successful Launches by Site')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, names='class result', title='Success vs Failed Launches at {}'.format(entered_site))
    return fig


# Callback function for `site-dropdown` and `payload-slider`
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def update_scatter_chart(entered_site, payload):
    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= payload[0]) & (spacex_df['Payload Mass (kg)'] <= payload[1])]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title='Payload vs Class')
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title='Payload vs Class at {}'.format(entered_site))
    return fig


if __name__ == '__main__':
    app.run_server()